In [ ]:
!pip install datasets==1.18.3
!pip install transformers==4.11.3

!pip install torchaudio
!pip install librosa
!pip install jiwer

  Using cached transformers-4.11.3-py3-none-any.whl (2.9 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects
  Using cached jiwer-3.0.3-py3-none-any.whl (21 kB)
  Using cached rapidfuzz-3.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)


In [ ]:
!pip install huggingface_hub==0.18.0
from huggingface_hub import notebook_login

notebook_login()



In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
# !pip install -U datasets

from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_9_0", "bn", split="train+validation", use_auth_token=True )
common_voice_test = load_dataset("mozilla-foundation/common_voice_9_0", "bn", split="test", use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch


In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


Map:   0%|          | 0/22251 [00:00<?, ? examples/s]

Map:   0%|          | 0/7748 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))


,client_id,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,f19ae940920663964f0c4c278b8c3c177d2a2c71558489816c31ad571cad175044a603c3b4bb82fa1e98f7a4cc2ee649d9d925f76ed87d066cf60f5259d1ef22,বর্গা বা ভাগচাষীরা এতে অংশ নেয়।,3,0,twenties,female,,bn,
1,1d399e4bb636369fe1b14770bd3a31db1e9f2ae3028b93ac24e92236afe512f24f6d1c0700a76df9261cf0ff40397413553bf666170eff7cae0b8c36676673c3,এছাড়াও পুরনো বল হাতে নিয়ে দ্রুতগতিসম্পন্ন বামহাতি অর্থোডক্স স্পিন বোলিং করতে পারতেন।,2,0,,,,bn,
2,62a5ab6a13925fc02f1a0f0cf41aa0fb48eb7bbfd7723284ced59dcbbbcb3a762bab318a37063f813db4b11198dc1faceb4c6c4aa1015245e5238062c111a8d4,একবছর পরই শ্রীলঙ্কার বিপক্ষে খেলার জন্য বাংলাদেশ দলের সদস্য মনোনীত হন।,2,0,twenties,female,,bn,
3,424c16a8450e8b742c1380d9d38d919d880a36379e8c7e09126a2a8411022bc42cbae339c1d8dfd42695fb073ca783ed5e86b8a2531d11c4491259ab1378e445,অনুশীলন দলের উদ্দেশ্য ছিল সশস্ত্র বিপ্লবের মাধ্যমে ভারত থেকে ব্রিটিশ শাসনের উচ্ছেদ।,4,2,twenties,male,Indian,bn,
4,5e991d71e29bc02e7b7498860c064037d96964b8df39e351fd6ca535c50e98f651445a02ebe89998246d514cc109e40adb7d1802f0f99a0471f93726235584c9,এছাড়াও দলের প্রয়োজনে মাঝেমধ্যে ডানহাতি অফ ব্রেক বোলিং করেন।,2,1,,,,bn,
5,2e16c841d9165def19112ba4d7f04e3736e4db2f3f29ef16ca4117176432b79c17bb1f102dc8c09d57ffc4209e61d1f0306ed41dc49a01be70458382463601be,তিনি মার্কস ইনস্টিটিউট অব উচ্চশিক্ষায় ইসলাম ও মুসলমানদের উপর এমএ কোর্স প্রশিক্ষণ দিয়েছিলেন।,2,0,,,,bn,
6,f19ae940920663964f0c4c278b8c3c177d2a2c71558489816c31ad571cad175044a603c3b4bb82fa1e98f7a4cc2ee649d9d925f76ed87d066cf60f5259d1ef22,বছরে একবার পাতা ঝরে।,2,0,twenties,female,,bn,
7,c7b89a481d3c2e657f21b55cfbd93bd345546201c58ca39b811de6cdec6efc6110f7ce78a81076747bc3ae5e1ff2dea436d0f153c7ac2f124f1b6896870f94bb,তিনি বলেছিলেন তখন গুগল ছিল খুবই ছোট একটি কোম্পানি।,2,1,,,,bn,
8,192506d7b2ce556dbcb4b64f0f99750a936415dc2a632a5ecbeff604a2e4bf5ea114a64f95c581d717dc84c4f30070ec9260c49deb3151439619750e9d0eb2a7,যিনি বাস্তববাদী শৈলীর একজন জার্মান চিত্রশিল্পী।,2,0,,,,bn,
9,bf3b21b19ada09d4f920065bb34dbf30d81b4f949881a349f55815b1f1038837f26207d9e580ba6ae5141d5cf5fb5cd02d341f66a8e9964d74eb3a94cca4df17,তিনি তার প্রেমিক অভিনেতা সায়েদ জামান শাওনএর সঙ্গে বিবাহ বন্ধনে আবদ্ধ হয়েছেন।,10,4,,,,bn,


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/22251 [00:00<?, ? examples/s]

Map:   0%|          | 0/7748 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))


In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


{' ': 0,
 '/': 1,
 'a': 2,
 'e': 3,
 'g': 4,
 'l': 5,
 'o': 6,
 'p': 7,
 'y': 8,
 '।': 9,
 '॥': 10,
 'ঁ': 11,
 'ং': 12,
 'ঃ': 13,
 'অ': 14,
 'আ': 15,
 'ই': 16,
 'ঈ': 17,
 'উ': 18,
 'ঊ': 19,
 'ঋ': 20,
 'এ': 21,
 'ঐ': 22,
 'ও': 23,
 'ঔ': 24,
 'ক': 25,
 'খ': 26,
 'গ': 27,
 'ঘ': 28,
 'ঙ': 29,
 'চ': 30,
 'ছ': 31,
 'জ': 32,
 'ঝ': 33,
 'ঞ': 34,
 'ট': 35,
 'ঠ': 36,
 'ড': 37,
 'ঢ': 38,
 'ণ': 39,
 'ত': 40,
 'থ': 41,
 'দ': 42,
 'ধ': 43,
 'ন': 44,
 'প': 45,
 'ফ': 46,
 'ব': 47,
 'ভ': 48,
 'ম': 49,
 'য': 50,
 'র': 51,
 'ল': 52,
 'শ': 53,
 'ষ': 54,
 'স': 55,
 'হ': 56,
 '়': 57,
 'া': 58,
 'ি': 59,
 'ী': 60,
 'ু': 61,
 'ূ': 62,
 'ৃ': 63,
 'ে': 64,
 'ৈ': 65,
 'ো': 66,
 'ৌ': 67,
 '্': 68,
 'ৎ': 69,
 'ড়': 70,
 'য়': 71,
 'ৰ': 72,
 '—': 73,
 '’': 74,
 '‚': 75}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]


In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)


78

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [ ]:
# !pip install transformers

from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-bn-colab"


In [ ]:
tokenizer.push_to_hub(repo_name)


CommitInfo(commit_url='https://huggingface.co/pro10/wav2vec2-large-xls-r-300m-bn-colab/commit/2f40eb8daa06dd6796c367da0dc74613ce1801fc', commit_message='Upload tokenizer', commit_description='', oid='2f40eb8daa06dd6796c367da0dc74613ce1801fc', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [ ]:
common_voice_train[0]["path"]


'/root/.cache/huggingface/datasets/downloads/extracted/40d8327d140fdd493c8c19e4b06cb03a3b311176b16e1b826895bc07bae68c09/cv-corpus-9.0-2022-04-27/bn/clips/common_voice_bn_30614352.mp3'

In [ ]:
common_voice_train[0]["audio"]


{'path': '/root/.cache/huggingface/datasets/downloads/extracted/40d8327d140fdd493c8c19e4b06cb03a3b311176b16e1b826895bc07bae68c09/cv-corpus-9.0-2022-04-27/bn/clips/common_voice_bn_30614352.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.25871447e-05, -1.87064288e-05, -1.10539986e-05]),
 'sampling_rate': 48000}

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))


In [ ]:
common_voice_train[0]["audio"]


{'path': '/root/.cache/huggingface/datasets/downloads/extracted/40d8327d140fdd493c8c19e4b06cb03a3b311176b16e1b826895bc07bae68c09/cv-corpus-9.0-2022-04-27/bn/clips/common_voice_bn_30614352.mp3',
 'array': array([ 1.58818678e-22,  2.64697796e-23, -1.05879118e-22, ...,
         4.94913547e-06,  5.28711826e-06, -6.50915899e-06]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


আর কন্যা সারাহ্ মান্নান হোসাইন যুক্তরাষ্ট্র প্রবাসী।


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])


Target text: এছাড়া তিনি সুখদুঃখদ্বয়পরিত্যাগদৃষ্টি নামক একটি গ্রন্থ রচনা করেন।
Input array shape: (123840,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/22251 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/7748 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
wer_metric = load_metric("wer")


<ipython-input-36-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
# training starts
# !pip install transformers[torch]
# !pip install accelerate -U
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  # fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)


In [ ]:
trainer.train()
